![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [146]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your api key (hit enter): ")
)



Please enter your api key (hit enter):  ········


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [147]:
model_id = "meta-llama/llama-3-3-70b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [148]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [149]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [150]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [151]:
chat_messages = [];


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [152]:
question = input("Question: ")
chat_messages.append({
    "role": "system",
    "content": f"""You are a SQL generation assistant. Your task is to convert natural language questions into accurate SQL queries based on the given database schema.

Always follow these rules:
1. Use the exact table and column names from the schema.
2. Do not guess or create new columns or tables.
3. Use single quotes for string values.
4. Keep the SQL syntactically correct and executable.
5. Use proper SQL functions (e.g., COUNT, AVG, MAX, MIN) when the question asks for totals, averages, or extremes.
6. Use WHERE clauses to filter data, and ORDER BY or LIMIT when required.
7. If multiple interpretations are possible, choose the most straightforward SQL query.
8. Always assume the data source is a standard relational SQL database.
9. Return only the SQL query. Do not include explanations, comments, or summaries or styles or codes or anything.

Schema:
Table: employee
- ID (INTEGER)
- FIRST_NAME (VARCHAR)
- LAST_NAME (VARCHAR)
- SALARY (DECIMAL)
- POSITION (VARCHAR)

Examples:

Q: Who earns the highest salary?
A: SELECT FIRST_NAME, LAST_NAME FROM employee ORDER BY SALARY DESC LIMIT 1;

Q: What is the average salary of all employees?
A: SELECT AVG(SALARY) FROM employee;

Q: List the names of all employees who work as 'Analyst'.
A: SELECT FIRST_NAME, LAST_NAME FROM employee WHERE POSITION = 'Analyst';

Q: How many employees work as 'Engineer'?
A: SELECT COUNT(*) FROM employee WHERE POSITION = 'Engineer';

Q: What is the total salary paid to all Managers?
A: SELECT SUM(SALARY) FROM employee WHERE POSITION = 'Manager';

Q: List employees whose salary is greater than 80000.
A: SELECT FIRST_NAME, LAST_NAME FROM employee WHERE SALARY > 80000;

Q: Show all employees sorted by salary in descending order.
A: SELECT FIRST_NAME, LAST_NAME, SALARY FROM employees ORDER BY SALARY DESC;

Q: What is the minimum salary among Directors?
A: SELECT MIN(SALARY) FROM employee WHERE POSITION = 'Director';

Start generating SQL queries now."""
})

chat_messages.append({"role": "user", "content": question})
sql_query = model.chat(messages=chat_messages)
sql_text = sql_query['choices'][0]['message']['content'].strip()
print(sql_text)

Question:  What is the last name of the employee with the least salary


SELECT LAST_NAME FROM employee ORDER BY SALARY ASC LIMIT 1;


In [153]:
!pip install ibm_db

In [ ]:
import ibm_db
import pandas as pd # Useful for displaying query results

db2_credentials = {
    'database': input("Enter DB2 database name: "),
    'hostname': input("Enter DB2 hostname: "),
    'port': input("Enter DB2 port: "),
    'username': input("Enter DB2 username: "),
    'password': getpass.getpass("Enter DB2 password: "),
    'security': input("Enter DB2 security type (e.g., SSL): ")
}

In [ ]:
try:
    database = db2_credentials['database']
    hostname = db2_credentials['hostname']
    port = db2_credentials['port']
    uid = db2_credentials['username']
    pwd = db2_credentials['password']
    schema_name = 'GCE'

    
    security = db2_credentials.get('security', '') # Safely get 'security' or default to empty string

    # Build the connection string
    conn_string = (
        f"DATABASE={database};HOSTNAME={hostname};PORT={port};"
        f"PROTOCOL=TCPIP;UID={uid};PWD={pwd};"
    )
    if security.upper() == 'SSL':
        conn_string += "SECURITY=SSL;"

    print("\nAttempting to connect to DB2...")
    db2_conn = ibm_db.connect(conn_string, "", "") # username and password are part of the conn_string

    if db2_conn:
        print("Connection to DB2 successful!")

        # --- IMPORTANT: Set the current schema for the session ---
        set_schema_query = f"SET CURRENT SCHEMA = '{schema_name}';"
        print(f"Setting current schema: {set_schema_query}")
        ibm_db.exec_immediate(db2_conn, set_schema_query)
        print(f"Current schema set to '{schema_name}'.")
        # --- End of schema setting ---
    else:
        print("Failed to connect to DB2.")

except Exception as e:
    print(f"Error during connection: {e}")
    db2_conn = None # Ensure db2_conn is None if connection fails


Attempting to connect to DB2...
Connection to DB2 successful!
Setting current schema: SET CURRENT SCHEMA = 'GCE';
Current schema set to 'GCE'.


In [156]:
if db2_conn and sql_text:
    try:
        print(f"\nExecuting query: \n{sql_text}\n")

        # Execute the query
        stmt = ibm_db.exec_immediate(db2_conn, sql_text)

        # Fetch results
        results = []
        row = ibm_db.fetch_assoc(stmt)
        while row:
            results.append(row)
            row = ibm_db.fetch_assoc(stmt)

        # Display results using Pandas for better readability
        if results:
            df_results = pd.DataFrame(results)
            print("Query Results:")
            print(df_results)
        else:
            print("No results returned for the query.")

    except Exception as e:
        print(f"Error executing SQL query: {e}")
        # Detailed error for DB2
        if db2_conn:
            print(f"DB2 Error Code: {ibm_db.conn_error(db2_conn)}")
            print(f"DB2 Error Message: {ibm_db.conn_errormsg(db2_conn)}")
    finally:
        # Close the statement resource
        if 'stmt' in locals() and stmt:
            ibm_db.free_stmt(stmt)
        if db2_conn:
            ibm_db.close(db2_conn)
            print("\nDB2 connection closed.")
else:
    print("Database connection not established or SQL query is empty. Cannot execute.")


Executing query: 
SELECT LAST_NAME FROM employee ORDER BY SALARY ASC LIMIT 1;

Query Results:
  LAST_NAME
0       Lee

DB2 connection closed.


In [ ]:

results_text = df_results.to_string(index=False) # index=False prevents printing the DataFrame index
print("\nFormatted Query Results for LLM:")
print(results_text)


Formatted Query Results for LLM:
LAST_NAME
      Lee


In [ ]:
answer_chat_messages = []

# Add a system prompt for answer generation
answer_chat_messages.append({
    "role": "system",
    "content": f"""You are an intelligent assistant that provides concise natural language answers.
Given the original user question and the retrieved database results, formulate a clear and direct answer.
Do not include the SQL query or schema information in your answer.
"""
})

# Add the user's input, combining the original question and the query results
answer_chat_messages.append({
    "role": "user",
    "content": f"""Original User Question: "{question}"

Database Query Results:
{results_text}

Please provide the natural language answer to the original question."""
})

# 3. Call model.chat() again with the new messages
generated_answer_response = model.chat(messages=answer_chat_messages)

# 4. Extract the final answer
try:
    #print(generated_answer_response)
    final_answer = generated_answer_response['choices'][0]['message']['content']
    print("\n--- Final Natural Language Answer ---")
    print(final_answer)
except (AttributeError, IndexError, KeyError) as e:
    print(f"Error extracting final answer from chat response: {e}")
    print("Response structure might be unexpected. Full response:")
    print(generated_answer_response.to_dict()) # Convert to dict to inspect
    final_answer = "I'm sorry, I couldn't generate a clear answer."

{'id': 'chatcmpl-757cc4b5-643a-4a4a-8cca-c473988cee0a---3c048f830f37af6cce3b87419dc163ab', 'object': 'chat.completion', 'model_id': 'meta-llama/llama-3-3-70b-instruct', 'model': 'meta-llama/llama-3-3-70b-instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The last name of the employee with the least salary is Lee.'}, 'finish_reason': 'stop'}], 'created': 1750843812, 'model_version': '3.3.0', 'created_at': '2025-06-25T09:30:12.416Z', 'usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131}, 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}

--- Final Natural Language Answer ---
The last name of the employee with the